# Model development with SciKit Learn library

## Import external libraries and load external file into data frame

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/Mini Project 2 ECSE 551 Team 2')

IMPORTANT: when any change is made to external py libraries, Factory Reset of Runtime is required!

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


In [ ]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import warnings
from sklearn.utils import shuffle
from scipy import special
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import Normalizer

from gensim.models.doc2vec import TaggedDocument

warnings.filterwarnings('ignore'), 
np.set_printoptions(precision=2)


In [ ]:
pd.options.display.max_colwidth = 100

In [ ]:
example_file_path = 'https://raw.githubusercontent.com/rgmartin/ECSE551/main/reddit/ExampleSubmissionRandom.csv'
train_file_path = 'https://raw.githubusercontent.com/rgmartin/ECSE551/main/reddit/train.csv'
test_file_path = 'https://raw.githubusercontent.com/rgmartin/ECSE551/main/reddit/test.csv'
corspus_file_path  = 'https://gist.githubusercontent.com/kunalj101/ad1d9c58d338e20d09ff26bcc06c4235/raw/1d2261e2276cbb0257a2ed6e2f1f4320464c7c07/corpus'

In [ ]:
train =  pd.read_csv(train_file_path, sep=',')
test = pd.read_csv(test_file_path, sep= ',')
example = pd.read_csv(example_file_path, sep= ',')

In [ ]:
#uniform_df = [train.loc[train.subreddit==i].sample(n=int(2000/8)) for i in train.subreddit.unique()]
#uniform_df = pd.concat(uniform_df, axis=0,ignore_index=True)
X_train = train.body
y_train, target_labels = pd.factorize(train.subreddit)

In [ ]:
X_test = test.body

## Text processing

In [ ]:
from sklearn.feature_extraction import text

In [ ]:
my_stop_words = text.ENGLISH_STOP_WORDS

### Feature engineering

Count vectors as features

In [ ]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(X_train)

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(X_train)

TF-IDF vectors as features

In [ ]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(X_train)
xtrain_tfidf =  tfidf_vect.transform(X_train)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(X_train)
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(X_train)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(X_train)
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(X_train) 

Word embedding

In [ ]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('data/wiki-news-300d-1M.vec')):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

#### Clean text

In [ ]:
import re

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z ]')

def text_prepare(text):
    """
    text: a string
        return: modified initial string
    """
    lower = text.lower() # lowercase text    
    space_replace = REPLACE_BY_SPACE_RE.sub(" ",lower) #replace REPLACE_BY_SPACE_RE symbols by space in text
    nosymb = BAD_SYMBOLS_RE.sub("",space_replace) # delete symbols which are in BAD_SYMBOLS_RE from text    
    text = re.sub(r"\s*\b(?:{})\b".format("|".join(my_stop_words)), "", nosymb) # delete STOPWORDS

    return re.sub(r" {2,}", " ", text.strip())

In [ ]:
X_train = X_train.apply(text_prepare)
X_test = X_test.apply(text_prepare)

In [ ]:
X_train.head(10)

#### Failed attempts

In [ ]:
from nltk.stem.porter import PorterStemmer
def stemm_text(my_text):
  my_tokens = my_text.split()
  porter = PorterStemmer()
  stemmed_tokens = []
  for token in my_tokens:
    stemmed_tokens.append(porter.stem(token))
  separator = ' '
  return separator.join(stemmed_tokens)

# It didn't increase the accuracy!!

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemm_text(my_text):
  my_tokens = my_text.split()  
  stemmed_tokens = []
  for token in my_tokens:
    stemmed_tokens.append(lemmatizer.lemmatize(token))
  separator = ' '
  return separator.join(stemmed_tokens)

# it didn't increase accuracy

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)
class New_LemmaTokenizer:
     def __init__(self):
       self.wnl = WordNetLemmatizer()
     def __call__(self, doc):
       return [self.wnl.lemmatize(t,pos =get_wordnet_pos(t)) for t in word_tokenize(doc) if t.isalpha()]
# it didn't increase accuracy

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## Algorithms implementation

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import Normalizer

text_clf_logreg = Pipeline([
                ('vect', CountVectorizer(ngram_range=(1,2))),                
                #('norm',Normalizer(copy = 'False')),
                ('tfidf', TfidfTransformer()),                
                ('clf', LogisticRegression(n_jobs=-1, C=100)),
               ])

In [ ]:
text_clf_logreg.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LogisticRegression(C=100, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
  

In [ ]:
np.mean(cross_val_score(text_clf_logreg, X_train, y_train, verbose = 10))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................
[CV] .................................... , score=0.897, total=  55.1s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   55.2s remaining:    0.0s


[CV] .................................... , score=0.905, total=  56.4s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.9min remaining:    0.0s


...

[CV] .................................... , score=0.897, total=  57.5s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  3.7min remaining:    0.0s


[CV] .................................... , score=0.895, total=  57.0s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  4.7min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  4.7min finished


0.8988080516241371

Confussion matrix

In [ ]:
from sklearn.model_selection import cross_val_predict
y_train_pred = cross_val_predict(text_clf_logreg, X_train, y_train, cv=5, verbose = 10)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.0min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  3.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  4.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  5.2min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  5.2min finished


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train, y_train_pred)

array([[1873,   12,   23,   10,   26,   11,   30,    3],
       [  30, 1229,   12,    5,   12,    7,    9,    0],
       [  18,    2, 2305,   18,   15,    0,   11,   13],
       [  15,    5,   42, 1408,   28,   61,   26,   32],
       [  15,   10,   19,   25, 1983,    1,    7,    4],
       [  79,   10,   15,   82,   24,  506,   67,    1],
       [  57,    6,   93,   11,   17,   17,  810,    5],
       [   5,    1,   18,   75,   20,    2,   10,  296]])

Creation of submission csv

In [ ]:
submission = test.copy()
prediction = text_clf_logreg.predict(X_test)
submission['subreddit'] = target_labels[prediction]

,id,body,subreddit
0,0,I am a data scientist with two years of experience. I figured some people might get some from my...,datascience
1,1,Meh this episode was pretty lame and I can't imagine 11 more episodes of this already tired gig...,anime
2,2,I run a fantasy campaign where character advancement is tracked by the players performing achiev...,rpg
3,3,Did you make sure the colour depth was set to 10bit in your nvidia control panel? Sounds like an...,hardware
4,4,For comparison here's the useable space on the Xbox One for different drive sizes. * (a) 500GB...,hardware
...,...,...,...
2893,2893,you wouldn't believe how many people with this kind of learning are incapable of even applying h...,datascience
2894,2894,Hi I make a game called [The Elf Game](https://www.osrelfgame.com/store) that has a single page...,gamedev
2895,2895,>I’ll write a few lines of code but then stop and think “I’m not doing this right” or I’ll run i...,gamedev
2896,2896,A coworker and I are having a debate on the term All Wheel Drive. He has a Hyundai Santa Fe with...,cars


In [ ]:
submission = submission.drop(columns=['body'])
submission.to_csv('out.csv', index=False)

,id,subreddit
0,0,datascience
1,1,anime
2,2,rpg
3,3,hardware
4,4,hardware
...,...,...
2893,2893,datascience
2894,2894,gamedev
2895,2895,gamedev
2896,2896,cars


#### Auxiliary grid search

In [ ]:
parameters = {    
    'vect__ngram_range': [(1, 1)],
    'tfidf__use_idf': [True],
    'clf__C':[80,90,100,110,120,130,900],
    'clf__penalty':["l2"]
}
gs_clf_logreg=GridSearchCV(text_clf_logreg, parameters, cv=5, n_jobs=-1)

In [ ]:
gs_clf_logreg = gs_clf_logreg.fit(X_train, y_train)

In [ ]:
gs_clf_logreg.best_score_

0.8975128266481208

In [ ]:
gs_clf_logreg.best_params_

{'clf__C': 100,
 'clf__penalty': 'l2',
 'tfidf__use_idf': True,
 'vect__ngram_range': (1, 1)}

#### Grid search for logistic regression.. no need to run it again (takes a long time), I already put the best parameters on the above cell.

In [ ]:
parameters = {    
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False),
    'clf__C':np.logspace(-3,3,7),
    'clf__penalty':["l1","l2"]
}
gs_clf_logreg=GridSearchCV(text_clf_logreg, parameters, cv=5, n_jobs=-1)

In [ ]:
gs_clf_logreg = gs_clf_logreg.fit(X_train, y_train)

In [ ]:
gs_clf_logreg.best_score_

0.8933688297728809

In [ ]:
gs_clf_logreg.best_params_

{'clf__C': 100.0,
 'clf__penalty': 'l2',
 'tfidf__use_idf': True,
 'vect__ngram_range': (1, 1)}

### Word2Vec with Logistic Regression

In [ ]:
import nltk
from nltk.corpus import stopwords
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

In [ ]:
train_tagged = train.apply(  lambda r: TaggedDocument(words=tokenize_text(r['body']), tags = r['subreddit']), axis = 1 )

In [ ]:
import multiprocessing

cores = multiprocessing.cpu_count()
cores

2

In [ ]:
from gensim.models import Doc2Vec
from tqdm import tqdm

model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

100%|██████████| 11582/11582 [00:00<00:00, 1461005.38it/s]


In [ ]:
from sklearn import utils


for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 11582/11582 [00:00<00:00, 1469090.90it/s]


In [ ]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [ ]:
y_train_w2v, X_train_w2v = vec_for_learning(model_dbow, train_tagged)

In [ ]:
from sklearn.linear_model import LogisticRegression

text_clf_logreg_w2v = Pipeline([('vect', CountVectorizer(ngram_range= (1, 1))),
                ('tfidf', TfidfTransformer(use_idf = True)),
                ('clf', LogisticRegression(n_jobs=1, C=100, penalty='l2')),
               ])

In [ ]:
np.mean(cross_val_score(text_clf_logreg_w2v, X_train, y_train, cv=5))

0.8933688297728809

### Naive Bayes

Import required models from sklearn

In [ ]:
from sklearn.naive_bayes import MultinomialNB

Create pipeline

In [ ]:

text_clf_NB = Pipeline([
  ('vect', CountVectorizer(stop_words= my_stop_words)),
  ('norm', Normalizer()),
  ('tfidf', TfidfTransformer()),  
  ('clf', MultinomialNB()),
])

We can now train the model with a single command

In [ ]:
text_clf_NB.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=frozenset({'a', 'about', 'above',
                                                       'across', 'after',
                                                       'afterwards', 'again',
                                                       'against', 'all',
                                                       'a...
                                                       'another', 'any',
                                                       'anyhow', 'anyone',
                             

Compute mean accuracy with 5-fold cross validation

In [ ]:

np.mean(cross_val_score(text_clf_NB, X_train, y_train, cv =5))

0.7584182169337844

### Support Vector Machine

In [ ]:
from sklearn.linear_model import SGDClassifier
text_clf_SVM = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
      alpha=1e-3, random_state=42,
      max_iter=5, tol=None)),
])

In [ ]:
text_clf_SVM

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                ('clf',
                 SGDClassifier(alpha=0.001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                      

In [ ]:
text_clf_SVM.fit(X_train,y_train)
np.mean(cross_val_score(text_clf_SVM, X_train, y_train, cv=5))

0.8532202844038543

#### Parameter tuning using grid search

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False),
    'clf__alpha': (1e-2, 1e-3),
}

In [ ]:
gs_clf_SVM = GridSearchCV(text_clf_SVM, parameters, cv=5, n_jobs=-1)

In [ ]:
gs_clf_SVM = gs_clf_SVM.fit(X_train, y_train)

In [ ]:
gs_clf_SVM.best_score_

0.8532202844038543

In [ ]:
gs_clf_SVM.best_params_

{'clf__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

text_clf_rnd_frest = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0)),
               ])

In [ ]:
text_clf_rnd_frest.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=3, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                   

In [ ]:
np.mean(cross_val_score(text_clf_rnd_frest, X_train, y_train, cv=5))

0.3944949211467691